In [17]:
import helper_functions.data_helpers as help
import helper_functions.variables as var

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

from bokeh.models import ColumnDataSource

In [2]:
master_df = help.get_master_df()
master_df.describe()

,Total Consumption,Total Production,Nuclear production,Industrial Cogeneration,Cogenation of district heat,Hydro production,Wind production,Net import/export,Reserve power plants and small-scale production
count,177737.000000,177737.000000,177737.000000,177737.000000,177737.000000,177738.000000,177736.000000,177737.000000,177737.000000
mean,9030.647220,7634.697385,2751.373605,1105.948553,931.915729,1417.934812,1253.396409,-1394.359085,111.639214
std,1344.490868,1283.271153,499.525105,179.080729,581.501809,523.292079,915.652664,846.245255,133.498226
min,6116.800000,4073.000000,1800.700000,328.150000,113.500000,312.160000,2.000000,-4529.500000,11.000000
25%,8034.700000,6689.000000,2654.000000,986.500000,480.000000,958.162500,475.017500,-1908.700000,41.000000
50%,8789.000000,7611.600000,2771.800000,1096.500000,887.660000,1463.200000,1029.000000,-1481.400000,62.000000
75%,10030.000000,8503.900000,2801.000000,1213.300000,1316.700000,1870.700000,1918.000000,-911.340000,103.000000
max,13846.000000,12644.000000,4385.300000,1633.200000,2900.700000,2503.300000,4269.000000,1875.900000,686.000000


In [31]:
#ColumnDataSource(data={"angle" : [1,2,3]})
angles_df = pd.DataFrame((master_df*np.array([1.0]*9)).sum(axis=0)[var.to_sum])
angles_df["angle"] = (angles_df/angles_df.sum())*  2* np.pi
angles_df.columns = ["Value", "angle"]
angles_df["color"] = var.to_sum_colors
angles_df.T

,Nuclear production,Industrial Cogeneration,Cogenation of district heat,Hydro production,Wind production,Net import/export,Reserve power plants and small-scale production
Value,489020890.5,196567977.93,165635905.84,252020897.69,222773664.1321,-247829200.62291,19842418.984
angle,2.798286,1.124805,0.947805,1.442119,1.27476,-1.418134,0.113543
color,DarkGreen,IndianRed,SandyBrown,SlateBlue,Khaki,darkmagenta,Tomato


In [7]:
def get_one_average_day_from_df(df: pd.DataFrame, first_date: datetime, current_day_number: datetime, average_over: int, multipliers: np.array(int)) -> pd.DataFrame:
    if average_over < 2:
        return get_one_day_from_df(df, first_date, current_day_number)

    original_date = first_date + timedelta(days = current_day_number%365)
    next_date = original_date + timedelta(days = 1)

    this_day= df[(df.index >= original_date) & (df.index < next_date)]

    for i in range(1, average_over):
        current_date = first_date + timedelta(days = (current_day_number + i)%365)
        next_date = current_date + timedelta(days = 1)
        next_day = df[(df.index >= current_date) & (df.index < next_date)]
        next_day.index = next_day.index + pd.DateOffset((original_date - current_date).days)
        this_day = this_day.combine(next_day, lambda a, b: a+b)

    return pd.DataFrame(this_day/average_over) * multipliers

print()
get_one_average_day_from_df(master_df, master_df.index[0], 362, 10, np.array([1.0]*9))


,Total Consumption,Total Production,Nuclear production,Industrial Cogeneration,Cogenation of district heat,Hydro production,Wind production,Net import/export,Reserve power plants and small-scale production
start_time,,,,,,,,,
2022-12-29 00:01:00+00:00,9920.74,7900.18,3045.25,1153.287,1352.143,746.940,1477.852,-2023.0176,126.4000
2022-12-29 00:04:00+00:00,9931.30,7900.31,3044.79,1153.263,1350.562,749.910,1475.347,-2026.9270,126.5000
2022-12-29 00:07:00+00:00,9931.11,7891.43,3045.24,1149.604,1348.289,741.401,1478.549,-2039.9950,126.9000
2022-12-29 00:10:00+00:00,9921.79,7882.46,3046.55,1146.582,1344.257,737.404,1480.560,-2037.7050,127.0000
2022-12-29 00:13:00+00:00,9905.59,7873.42,3046.25,1146.480,1342.666,726.021,1486.338,-2035.2590,126.6000
...,...,...,...,...,...,...,...,...,...
2022-12-29 23:46:00+00:00,10017.29,8202.00,3106.94,1130.210,1360.225,829.920,1642.460,-1815.4623,131.0000
2022-12-29 23:49:00+00:00,10015.89,8203.89,3106.84,1132.307,1361.613,825.429,1646.650,-1802.8942,131.0000
2022-12-29 23:52:00+00:00,9980.98,8204.24,3107.04,1135.050,1359.930,820.512,1650.695,-1769.3988,131.0000


In [61]:
df = master_df
first_date = df.index[0]
current_day_number = 364
average_over = 2

original_date = first_date + timedelta(days = current_day_number%365)
next_date = original_date + timedelta(days = 1)

this_day= df[(df.index >= original_date) & (df.index < next_date)]
print(this_day[["Total Consumption", "Total Production"]].head())

for i in range(1, average_over):
    current_date = first_date + timedelta(days = (current_day_number + i)%365)
    next_date = current_date + timedelta(days = 1)
    next_day = df[(df.index >= current_date) & (df.index < next_date)]
    next_day.index = next_day.index + pd.DateOffset((original_date - current_date).days)
    this_day = this_day.combine(next_day, lambda a, b: a+b)
    print(next_day[["Total Consumption", "Total Production"]].head())


#return this_day/average_over
this_day.shape

                           Total Consumption  Total Production
start_time                                                    
2022-12-31 00:01:00+00:00             9062.3            9178.5
2022-12-31 00:04:00+00:00             9076.3            9179.7
2022-12-31 00:07:00+00:00             9005.0            9249.4
2022-12-31 00:10:00+00:00             9062.7            9290.1
2022-12-31 00:13:00+00:00             9074.0            9333.5
                           Total Consumption  Total Production
start_time                                                    
2022-12-31 00:01:00+00:00             9849.2            7607.6
2022-12-31 00:04:00+00:00             9881.7            7579.7
2022-12-31 00:07:00+00:00             9904.1            7550.7
2022-12-31 00:10:00+00:00             9874.0            7528.8
2022-12-31 00:13:00+00:00             9827.7            7539.0


(480, 8)

In [60]:
#datetime.fromtimestamp(current_date)
#type(current_date.to_pydatetime())
(original_date - current_date).days
pd.DateOffset((original_date - current_date).days)

<364 * DateOffsets>

In [125]:
a = pd.DataFrame({"v": [1,2,3], "n": [1,2,3]})
b = pd.DataFrame({"v": [1,2,3], "n": [1,2,3]})


#print(this_day.iloc[30])
#next_day.index = next_day.index - timedelta(days=1)
#this_day.combine(next_day, lambda a, b: a+b)
next_day

,Total Consumption,Total Production,Hydro production,Industrial Cogeneration,Net import/export,Cogenation of district heat,Nuclear production,Wind production
start_time,,,,,,,,
2021-12-31 00:01:00+00:00,10566.0,6881.0,722.61,1191.8,-3696.2,1479.8,2802.8,502.99
2021-12-31 00:04:00+00:00,10541.0,6869.7,708.84,1192.2,-3630.0,1478.3,2801.2,508.49
2021-12-31 00:07:00+00:00,10510.0,6900.3,718.81,1193.2,-3599.5,1479.5,2801.2,524.47
2021-12-31 00:10:00+00:00,10476.0,6910.3,714.00,1202.0,-3568.2,1476.0,2799.3,538.00
2021-12-31 00:13:00+00:00,10508.0,6909.8,712.34,1203.5,-3587.5,1469.8,2803.0,541.00
...,...,...,...,...,...,...,...,...
2021-12-31 23:46:00+00:00,10239.0,7622.0,456.53,1186.5,-2614.2,1325.7,2802.8,1713.00
2021-12-31 23:49:00+00:00,10235.0,7620.5,454.50,1191.0,-2612.2,1328.0,2801.0,1712.00
2021-12-31 23:52:00+00:00,10203.0,7620.8,450.00,1194.8,-2569.6,1327.0,2801.2,1711.90


In [81]:
def fix_master_df(df:pd.DataFrame) -> pd.DataFrame:
    i = 1
    while i < len(df):
        if (df.index[1] - df.index[0] != df.index[i] - df.index[i-1]):
            data = df.iloc[i-1]
            df.insert(pd.DataFrame(data, index=["start_time"]))
        i += 1

df = fix_master_df(df)

AttributeError: 'DataFrame' object has no attribute 'append'

datetime.timedelta(days=2)

dtype=datetime64[ns, UTC] from dataframe